Create external database

In [12]:
IF NOT EXISTS (SELECT * FROM master.dbo.sysdatabases WHERE name = 'externalDB')
    BEGIN
        CREATE DATABASE externalDB
END;

SELECT * FROM master.dbo.sysdatabases WHERE name = 'externalDB';
USE externalDB;
GO

(1 row affected)

Total execution time: 00:00:01.345

name,dbid,sid,mode,status,status2,crdate,reserved,category,cmptlevel,filename,version
externalDB,9,0x40342F955279EF45AECE10823C41B8CD,0,65536,1627389952,2020-11-30 11:28:17.290,1900-01-01 00:00:00.000,0,150,/var/opt/mssql/data/externalDB.mdf,904


Create master key

In [13]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Password1234';

Commands completed successfully.

Total execution time: 00:00:00.045

Create external data source

In [14]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://controller-svc/default');

Commands completed successfully.

Total execution time: 00:00:01.002

Create external table

In [16]:
BEGIN TRY
    BEGIN TRANSACTION T3ac3ae894a2e4b7fab8d07ea5beb86d
        USE [externalDB];
        CREATE EXTERNAL FILE FORMAT [monthlyArrivals]
            WITH (FORMAT_TYPE = DELIMITEDTEXT, FORMAT_OPTIONS (FIELD_TERMINATOR = N',', STRING_DELIMITER = N'"', FIRST_ROW = 2));
        CREATE EXTERNAL TABLE [dbo].[monthlyArrivals]
        (
            [month] nvarchar(50) NOT NULL,
            [Total_Aircraft_Arrivals_And_Departures] smallint NOT NULL,
            [Total_Passengers] int NOT NULL
        )
        WITH (LOCATION = N'/Airport_CSV_Data/civil-aircraft-arrivals-departures-and-passengers-changi-airport-monthly-modified.csv', DATA_SOURCE = [SqlStoragePool], FILE_FORMAT = [monthlyArrivals]);
    COMMIT TRANSACTION T3ac3ae894a2e4b7fab8d07ea5beb86d
END TRY
BEGIN CATCH
    IF @@TRANCOUNT > 0
        ROLLBACK TRANSACTION T3ac3ae894a2e4b7fab8d07ea5beb86d
    DECLARE @ErrorMessage NVARCHAR(max) = ERROR_MESSAGE();
    DECLARE @ErrorSeverity INT = ERROR_SEVERITY();
    DECLARE @ErrorState INT = ERROR_STATE();
    RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);
END CATCH;


Commands completed successfully.

Total execution time: 00:00:00.281

Query data from table

In [17]:
SELECT TOP (1000) [month]
      ,[Total_Aircraft_Arrivals_And_Departures]
      ,[Total_Passengers]
  FROM [externalDB].[dbo].[monthlyArrivals]

: Msg 105082, Level 16, State 1, Line 1
105082;Generic ODBC error: [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Bulk load data conversion error (overflow) for row 457, column 2 (Total_Aircraft_Arrivals_And_Departures). .

OLE DB provider "MSOLEDBSQL" for linked server "(null)" returned message "Unspecified error".

: Msg 7421, Level 16, State 2, Line 1
Cannot fetch the rowset from OLE DB provider "MSOLEDBSQL" for linked server "(null)". .

Total execution time: 00:00:00.912

month,Total_Aircraft_Arrivals_And_Departures,Total_Passengers


 Drop the table and database (for cleanup purposes)

In [7]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[monthlyArrivals]') AND type in (N'U'))
DROP EXTERNAL TABLE [dbo].[monthlyArrivals]
GO

Commands completed successfully.

Total execution time: 00:00:00.018